In [1]:
import numpy as np 
import pandas as pd 
import tqdm
import string 
import seaborn as sns
import xgboost as xgb
import lightgbm as lgbm
from sklearn.svm import LinearSVC 
from sklearn.calibration import CalibratedClassifierCV 
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn import ensemble, metrics, model_selection, naive_bayes
import glob
import pandas as pd
from tensorflow import keras
import numpy as np
import os 
from sklearn.model_selection import StratifiedKFold, KFold
import matplotlib.pylab as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import TimeDistributed, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, BatchNormalization, concatenate, Input, ConvLSTM2D, Reshape, Conv3D, Flatten, LSTM, GRU, Dense,Dropout, Add
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Bidirectional, Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalMaxPool1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Sequential, load_model
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import re 

import nltk # for stopwords 
from nltk.corpus import stopwords
import gensim # for Word2Vec embeddings 
from sklearn.feature_extraction.text import CountVectorizer

from konlpy.tag import Mecab, Hannanum

In [2]:
train = pd.read_csv('news_train.csv')
test = pd.read_csv('news_test.csv') 
submission = pd.read_csv('sample_submission.csv') 

In [3]:
train.head()

,n_id,date,title,content,ord,info
0,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,1,0
1,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",2,1
2,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",3,1
3,NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,4,0
4,NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국적인 소비 붐 조성에 기여할 예정,1,0


In [4]:
test.head()

,n_id,date,title,content,ord,id
0,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가',1,NEWS00237_1
1,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",[이데일리 김대웅 기자] 반도체 장비 업체 코세스(089890)의 기술력이 마이크로...,2,NEWS00237_2
2,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가...,3,NEWS00237_3
3,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서...,4,NEWS00237_4
4,NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대","코세스는 반도체 장비를 제조, 판매하는 업체로 지난 2006년 11월 코스닥 시장에...",5,NEWS00237_5


In [5]:
mecab = Mecab()

In [6]:
train['title_tokenized'] = train['title'].apply(lambda x: mecab.morphs(x))

In [7]:
train['content_tokenized'] = train['content'].apply(lambda x: mecab.morphs(x))

In [8]:
test['title_tokenized'] = test['title'].apply(lambda x: mecab.morphs(x))
test['content_tokenized'] = test['content'].apply(lambda x: mecab.morphs(x))

In [9]:
# number of words in title
train['num_words_title'] = train['title_tokenized'].apply(lambda x: len(x)) 
test['num_words_title'] = test['title_tokenized'].apply(lambda x: len(x))   

# number of words in content 
train['num_words_content'] = train['content_tokenized'].apply(lambda x: len(x)) 
test['num_words_content'] = test['content_tokenized'].apply(lambda x: len(x)) 

In [10]:
# number of unique words in title 
train['title_num_unique'] = train['title_tokenized'].apply(lambda x: len(set(x))) 
test['title_num_unique'] = test['title_tokenized'].apply(lambda x: len(set(x))) 

# number of unique words in content 
train['content_num_unique'] = train['content_tokenized'].apply(lambda x: len(set(x))) 
test['content_num_unique'] = test['content_tokenized'].apply(lambda x: len(set(x))) 

In [11]:
# number of punctuations in text 
train['num_punctuations_title'] = train['title_tokenized'].apply(lambda x: len([c for c in x if c in string.punctuation]))
test['num_punctuations_title'] = test['title_tokenized'].apply(lambda x: len([c for c in x if c in string.punctuation]))

train['num_punctuations_content'] = train['content_tokenized'].apply(lambda x: len([c for c in x if c in string.punctuation]))
test['num_punctuations_content'] = test['content_tokenized'].apply(lambda x: len([c for c in x if c in string.punctuation]))

In [12]:
# [ 또는 (로 시작하는지에 대한 여부 
train["title_startswith_["]=train.title.apply(lambda x : str(x).startswith("[" ) or str(x).startswith("(")) 
train["content_startswith_["]=train.content.apply(lambda x : str(x).startswith("[" ) or str(x).startswith("(") ) 
test["title_startswith_["]=test.title.apply(lambda x : str(x).startswith("[" ) or str(x).startswith("(")) 
test["content_startswith_["]=test.content.apply(lambda x : str(x).startswith("[" ) or str(x).startswith("(") ) 


# ] 또는 )로 시작하는지에 대한 여부 
train["title_endswith_]"]=train.title.apply(lambda x : str(x).endswith("]" ) or str(x).endswith(")"))
train["content_endswith_]"]=train.content.apply(lambda x : str(x).endswith("]" ) or str(x).endswith(")") )
test["title_endswith_]"]=test.title.apply(lambda x : str(x).endswith("]" ) or str(x).endswith(")"))
test["content_endswith_]"]=test.content.apply(lambda x : str(x).endswith("]" ) or str(x).endswith(")") )


# ' 로 시작하는지에 대한 여부 
train["title_startswith_quote"]=train.title.apply(lambda x : str(x).startswith('"'))
train["content_startswith_quote"]=train.content.apply(lambda x : str(x).startswith('"'))
test["title_startswith_quote"]=test.title.apply(lambda x : str(x).startswith('"'))
test["content_startswith_quote"]=test.content.apply(lambda x : str(x).startswith('"'))


# '로 끝나는지에 대한 여부
train["title_endswith_quote"]=train.title.apply(lambda x : str(x).endswith('"'))
train["content_endswith_quote"]=train.content.apply(lambda x : str(x).endswith('"'))
test["title_endswith_quote"]=test.title.apply(lambda x : str(x).endswith('"'))
test["content_endswith_quote"]=test.content.apply(lambda x : str(x).endswith('"'))


# 숫자로 시작하는지에 대한 여부 
train["title_startswith_number"]=train.title.apply(lambda x : str(x)[0].isdigit())
train["content_startswith_number"]=train.content.apply(lambda x : str(x)[0].isdigit())
test["title_startswith_number"]=test.title.apply(lambda x : str(x)[0].isdigit())
test["content_startswith_number"]=test.content.apply(lambda x : str(x)[0].isdigit())


# 숫자로 끝나는지에 대한 여부 
train["title_endswith_number"]=train.title.apply(lambda x : str(x)[-1].isdigit())
train["content_endswith_number"]=train.content.apply(lambda x : str(x)[-1].isdigit())
test["title_endswith_number"]=test.title.apply(lambda x : str(x)[-1].isdigit())
test["content_endswith_number"]=test.content.apply(lambda x : str(x)[-1].isdigit())



In [13]:
# title length 
train["title_length"] = train['title'].apply(lambda x : len(x))
test["title_length"] = test['title'].apply(lambda x : len(x))

In [14]:
# content length 
train['content_length'] = train['content'].apply(lambda x: len(x))
test['content_length'] = test['content'].apply(lambda x: len(x))

In [15]:
# average length of word in title 
train['title_mean_length'] = train['title_tokenized'].apply(lambda x: np.mean([len(w) for w in x])) 
test['title_mean_length'] = test['title_tokenized'].apply(lambda x: np.mean([len(w) for w in x])) 

In [16]:
train.shape, test.shape

((118745, 29), (142565, 29))

### Extract Text Based Features

In [17]:
def mecab_tokenizer(text):  
    tokens_mecab = mecab.morphs(text) 
    return tokens_mecab

In [18]:
# used for singular value decomposition  
tfidf_vec = TfidfVectorizer(tokenizer = mecab_tokenizer, ngram_range = (1,3)) 
full_tfidf_title = tfidf_vec.fit_transform(train['title'].values.tolist())
train_tfidf_title = tfidf_vec.transform(train['title'].values.tolist()) 
test_tfidf_title = tfidf_vec.transform(test['title'].values.tolist())

In [19]:
full_tfidf_title.shape, train_tfidf_title.shape, test_tfidf_title.shape 

((118745, 81195), (118745, 81195), (142565, 81195))

In [20]:
# used for singular value decomposition  
tfidf_vec = TfidfVectorizer(tokenizer = mecab_tokenizer, ngram_range = (1,3)) 
full_tfidf_content = tfidf_vec.fit_transform(train['content'].values.tolist()) 
train_tfidf_content = tfidf_vec.transform(train['content'].values.tolist()) 
test_tfidf_content = tfidf_vec.transform(test['content'].values.tolist())

In [21]:
full_tfidf_content.shape, train_tfidf_content.shape, test_tfidf_content.shape 

((118745, 1237825), (118745, 1237825), (142565, 1237825))

In [22]:
svd_obj = TruncatedSVD(n_components = 20, algorithm = 'arpack') 
svd_obj.fit(full_tfidf_title)
train_svd_title = pd.DataFrame(svd_obj.transform(train_tfidf_title)) 
test_svd_title = pd.DataFrame(svd_obj.transform(test_tfidf_title))

In [23]:
train_svd_title.columns = ['svd_word_title_' + str(i) for i in range(20)] 
test_svd_title.columns = ['svd_word_title_' + str(i) for i in range(20)] 

train = pd.concat([train, train_svd_title], axis = 1) 
test = pd.concat([test, test_svd_title], axis = 1)  

del full_tfidf_title, train_tfidf_title, test_tfidf_title, train_svd_title, test_svd_title

In [24]:
svd_obj = TruncatedSVD(n_components = 20, algorithm = 'arpack') 
svd_obj.fit(full_tfidf_content) 
train_svd_content = pd.DataFrame(svd_obj.transform(train_tfidf_content)) 
test_svd_content = pd.DataFrame(svd_obj.transform(test_tfidf_content)) 

In [25]:
train_svd_content.columns = ['svd_word_content_' + str(i) for i in range(20)] 
test_svd_content.columns = ['svd_word_content_' + str(i) for i in range(20)] 

train = pd.concat([train, train_svd_content], axis = 1) 
test = pd.concat([test, test_svd_content], axis = 1)  

del full_tfidf_content, train_tfidf_content, test_tfidf_content, train_svd_content, test_svd_content

In [26]:
train.shape, test.shape 

((118745, 69), (142565, 69))

In [29]:
train.to_csv('train_no_data_leak.csv',index=False) 
test.to_csv('test_no_data_leak.csv',index=False)